In [8]:
import pandas as pd
import pandas as pd
from wgs_qc_utils.reader import read_variant_calls
import yaml
from single_cell.utils import inpututils
import os
import pysam
import numpy as np

In [2]:
union = pd.read_csv("UNION.csv", sep="\t")
union["normal_wgs_counts"] = union.normal_wgs_counts.apply(lambda x : [int(n) for n  in x.replace("(", "").replace(")", "").split(",") ]) 
union["tumour_wgs_counts"] = union.tumour_wgs_counts.apply(lambda x : [int(n) for n  in x.replace("(", "").replace(")", "").split(",") ]) 


/home/abramsd/miniconda3/envs/mypython3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
union

,chr,pos,ref,alt,dlp_museq_caller,dlp_museq_score,dlp_strelka_caller,dlp_strelka_score,dlp_datatype,normal_wgs_counts,tumour_wgs_counts
0,1,10492,C,T,True,PR=0.51,False,NaN,DLP,"[94, 6]","[320, 102]"
1,1,76106,T,G,True,PR=0.61,False,NaN,DLP,"[27, 0]","[56, 1]"
2,1,531854,T,G,True,PR=0.61,True,QSS=31,DLP,"[99, 8]","[163, 23]"
3,1,536195,T,C,True,PR=0.61,False,NaN,DLP,"[122, 0]","[138, 5]"
4,1,546732,C,T,True,PR=0.66,False,NaN,DLP,"[82, 1]","[172, 9]"
...,...,...,...,...,...,...,...,...,...,...,...
108125,X,154168172,G,T,False,NaN,True,QSS=23,DLP,"[54, 0]","[109, 0]"
108126,X,154388106,A,T,False,NaN,True,QSS=27,DLP,"[48, 2]","[71, 0]"
108127,X,154388171,C,A,False,NaN,True,QSS=28,DLP,"[51, 2]","[71, 1]"
108128,X,154388203,G,A,False,NaN,True,QSS=27,DLP,"[46, 2]","[75, 0]"


In [4]:
union["normal_ref"] = union.normal_wgs_counts.apply(lambda counts: counts[0]) 
union["normal_alt"] = union.normal_wgs_counts.apply(lambda counts: counts[1]) 

union["tumour_ref"] = union.tumour_wgs_counts.apply(lambda counts: counts[0]) 
union["tumour_alt"] = union.tumour_wgs_counts.apply(lambda counts: counts[1]) 

In [5]:
union = union.drop(["normal_wgs_counts", "tumour_wgs_counts"], axis=1) 

In [6]:
[int(n) for n  in "(94, 6)".replace("(", "").replace(")", "").split(",") ]

[94, 6]

In [7]:
union.to_csv("DLP_with_wgs_readcounts.csv", sep="\t", index=False, header=True)

In [42]:
most_sure = union[(union.dlp_museq_caller ==True )  & (union.dlp_strelka_caller ==True)]
most_sure["alt_count_pseudonormalized"] =  most_sure.tumour_alt/most_sure.normal_alt.replace(0, 1)


/home/abramsd/miniconda3/envs/mypython3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
import matplotlib.pyplot as plt

# plt.hist(most_sure.alt_count_pseudonormalized, bins=100)

most_sure_with_support = most_sure[most_sure.alt_count_pseudonormalized > 2.5]



len(most_sure[most_sure.alt_count_pseudonormalized > 2.5].index)/len(most_sure.index) * 100

27.220844515405783

In [49]:
from scipy.stats import ttest_ind
print(ttest_ind(most_sure.normal_alt, most_sure.tumour_alt))

Ttest_indResult(statistic=-42.65332190070126, pvalue=0.0)


In [9]:

wgs_museq_file = '/juno/work/shah/isabl_data_lake/analyses/24/72/2472/results/copynumber/SPECTRUM-OV-022_S1_LEFT_OVARY/titan/SPECTRUM-OV-022_S1_LEFT_OVARY_museq.vcf'
wgs_strelka_file = '/juno/work/shah/isabl_data_lake/analyses/24/65/2465/results/variants/SPECTRUM-OV-022_S1_LEFT_OVARY/SPECTRUM-OV-022_S1_LEFT_OVARY_strelka_snv_annotated.vcf.gz'
wgs_museq = read_variant_calls.read_with_tumour(wgs_museq_file)
wgs_strelka = read_variant_calls.read_with_tumour(wgs_strelka_file)

In [10]:
union

,chr,pos,ref,alt,dlp_museq_caller,dlp_museq_score,dlp_strelka_caller,dlp_strelka_score,dlp_datatype,normal_ref,normal_alt,tumour_ref,tumour_alt
0,1,10492,C,T,True,PR=0.51,False,NaN,DLP,94,6,320,102
1,1,76106,T,G,True,PR=0.61,False,NaN,DLP,27,0,56,1
2,1,531854,T,G,True,PR=0.61,True,QSS=31,DLP,99,8,163,23
3,1,536195,T,C,True,PR=0.61,False,NaN,DLP,122,0,138,5
4,1,546732,C,T,True,PR=0.66,False,NaN,DLP,82,1,172,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108125,X,154168172,G,T,False,NaN,True,QSS=23,DLP,54,0,109,0
108126,X,154388106,A,T,False,NaN,True,QSS=27,DLP,48,2,71,0
108127,X,154388171,C,A,False,NaN,True,QSS=28,DLP,51,2,71,1
108128,X,154388203,G,A,False,NaN,True,QSS=27,DLP,46,2,75,0


In [11]:
wgs_museq

,chr,pos,id,ref,alt,qual,filter,info,RC_normal,AC_normal,...,DP_normal,GT_normal,PL_normal,RC_tumour,AC_tumour,NI_tumour,ND_tumour,DP_tumour,GT_tumour,PL_tumour
0,1,10622,.,T,G,8.32,INDL,PR=0.85;TC=GTT,19,5,...,24,0/1,"28,0,255",3,3,0,5,6,0/1,"40,0,40"
1,1,14653,.,C,T,10.14,PASS,PR=0.90;TC=ACG,53,72,...,125,0/1,"255,0,255",13,23,0,0,36,0/1,"255,0,255"
2,1,14907,.,A,G,10.32,PASS,PR=0.91;TC=AAG,112,210,...,322,0/1,"255,0,255",49,47,0,0,96,0/1,"255,0,255"
3,1,14930,.,A,G,11.05,PASS,PR=0.92;TC=CAA,125,163,...,288,0/1,"255,0,255",47,43,0,0,91,0/1,"255,0,255"
4,1,16068,.,T,C,9.41,PASS,PR=0.89;TC=CTG,63,76,...,139,0/1,"255,0,255",22,23,0,0,45,0/1,"255,0,255"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256739,X,155252868,.,T,A,10.80,PASS,PR=0.92;TC=CTC,79,19,...,98,0/1,"255,0,255",11,13,0,0,24,0/1,"255,0,255"
2256740,X,155253354,.,A,G,9.76,PASS,PR=0.89;TC=CAT,76,69,...,145,0/1,"255,0,255",15,16,0,0,31,0/1,"255,0,255"
2256741,X,155255386,.,G,T,9.96,PASS,PR=0.90;TC=CGG,72,70,...,142,0/1,"255,0,255",24,18,0,0,42,0/1,"255,0,255"
2256742,X,155256972,.,C,T,11.72,PASS,PR=0.93;TC=ACT,136,65,...,201,0/1,"255,0,255",32,29,0,0,61,0/1,"255,0,255"


In [21]:
union_with_wgs_museq = wgs_museq[(wgs_museq.chr.isin(union.chr)) & (wgs_museq.pos.isin(union.pos)) 
                                 &(wgs_museq.alt.isin(union.alt))  & (wgs_museq.ref.isin(union.ref))]


union_with_wgs_strelka = wgs_strelka[(wgs_strelka.chr.isin(union.chr)) & (wgs_strelka.pos.isin(union.pos)) 
                                 &(wgs_strelka.alt.isin(union.alt))  & (wgs_strelka.ref.isin(union.ref))]

In [22]:
union_with_wgs_strelka

,chr,pos,id,ref,alt,qual,filter,info,DP_normal,FDP_normal,...,GU_normal,TU_normal,DP_tumour,FDP_tumour,SDP_tumour,SUBDP_tumour,AU_tumour,CU_tumour,GU_tumour,TU_tumour
4,1,565406,.,C,T,.,.,QSS=22;TQSS=1;NT=ref;QSS_NT=22;TQSS_NT=1;SGT=C...,68,0,...,"0,1","0,0",96,1,0,0,"0,4","89,94","0,2","6,8"
6,1,753699,.,C,T,.,.,QSS=137;TQSS=1;NT=ref;QSS_NT=92;TQSS_NT=1;SGT=...,48,0,...,"0,0","0,0",94,0,0,0,"0,0","28,28","0,0","66,66"
7,1,818490,.,T,A,.,.,QSS=43;TQSS=1;NT=ref;QSS_NT=43;TQSS_NT=1;SGT=T...,76,3,...,"0,0","73,85",112,2,0,0,"18,26","0,0","0,0","92,118"
18,1,1649076,.,C,T,.,.,QSS=18;TQSS=2;NT=ref;QSS_NT=18;TQSS_NT=2;SGT=C...,59,0,...,"0,0","0,1",71,0,0,0,"2,2","62,67","0,0","7,8"
20,1,1650622,.,C,G,.,.,QSS=33;TQSS=1;NT=ref;QSS_NT=33;TQSS_NT=1;SGT=C...,82,17,...,"0,1","0,0",102,30,0,0,"0,0","63,110","9,17","0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9642,X,151550466,.,A,G,.,.,QSS=64;TQSS=1;NT=ref;QSS_NT=64;TQSS_NT=1;SGT=A...,38,0,...,"0,0","0,0",94,0,0,0,"60,62","0,0","34,35","0,0"
9643,X,152034136,.,T,C,.,.,QSS=114;TQSS=1;NT=ref;QSS_NT=95;TQSS_NT=1;SGT=...,56,1,...,"0,0","55,56",149,0,0,0,"0,0","65,65","0,0","84,84"
9646,X,153166682,.,C,A,.,.,QSS=101;TQSS=1;NT=ref;QSS_NT=96;TQSS_NT=1;SGT=...,64,2,...,"0,0","0,0",124,3,0,0,"45,47","76,83","0,0","0,0"
9647,X,153360511,.,C,A,.,.,QSS=57;TQSS=1;NT=ref;QSS_NT=57;TQSS_NT=1;SGT=C...,32,0,...,"0,0","0,0",99,5,0,0,"41,43","53,57","0,0","0,0"


In [23]:
union["in_wgs_museq"] = [False] * len(union.index)
union.loc[union.index.isin(union_with_wgs_museq.index), 'in_wgs_museq'] = True

union["in_wgs_strelka"] = [False] * len(union.index)
union.loc[union.index.isin(union_with_wgs_strelka.index), 'in_wgs_strelka'] = True

In [25]:
union.in_wgs_strelka.unique()

array([False,  True])

In [27]:
union.to_csv("DLP_wgs_counts_with_wgs_concordance.csv", sep="\t", index=False, header=True)